1: Import Required Libraries

In [1]:
import cv2
from ultralytics import YOLO
import pandas as pd
import pyttsx3
import os
import time
from collections import defaultdict


ModuleNotFoundError: No module named 'ultralytics'

**2: Environment Configuration**

In [ ]:
# Fix OpenMP duplicate library issue
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


**3: Initialize Text-to-Speech Engine**

In [ ]:
try:
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    print("Text-to-Speech initialized.")
except Exception as e:
    print(f"TTS Error: {e}")
    engine = None


**4: Load YOLO Model and Class Names**

In [ ]:
model_path = "yolo11n.pt"

classNames = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat',
    'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear'
]

print("Loading YOLO model...")
model = YOLO(model_path)
print("Model loaded successfully.")


**5: Camera Initialization**

In [2]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

print("Camera started.")


RuntimeError: Cannot open camera

6: Video Writer (Save Output)

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(
    'output.avi',
    fourcc,
    20.0,
    (int(cap.get(3)), int(cap.get(4)))
)


**7: Data Storage & Counters**

In [ ]:
df = pd.DataFrame(columns=['Class', 'Confidence', 'Time'])

object_counter = defaultdict(int)
last_spoken = {}
speech_delay = 2  # seconds


**8: FPS & Confidence Trackbar**

In [ ]:
prev_time = 0

def nothing(x):
    pass

cv2.namedWindow("YOLO Real-Time Detection")
cv2.createTrackbar(
    "Confidence",
    "YOLO Real-Time Detection",
    40, 100,
    nothing
)


**9: Real-Time Detection Loop**

In [ ]:
print("Press 'Q' to quit")

while True:
    success, frame = cap.read()
    if not success:
        break

    # FPS calculation
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time) if prev_time != 0 else 0
    prev_time = curr_time

    conf_threshold = cv2.getTrackbarPos(
        "Confidence",
        "YOLO Real-Time Detection"
    ) / 100

    results = model(frame, stream=True)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            if cls >= len(classNames) or conf < conf_threshold:
                continue

            class_name = classNames[cls]
            label = f"{class_name} {conf:.2f}"

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
            )

            # Count objects
            object_counter[class_name] += 1

            # Save detection
            df.loc[len(df)] = [
                class_name,
                conf,
                time.strftime("%H:%M:%S")
            ]

            # Voice feedback
            if engine:
                last_time = last_spoken.get(class_name, 0)
                if curr_time - last_time > speech_delay:
                    engine.say(f"Detected {class_name}")
                    engine.runAndWait()
                    last_spoken[class_name] = curr_time

    # Display FPS
    cv2.putText(
        frame, f"FPS: {int(fps)}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2
    )

    # Display counters
    y_offset = 80
    for obj, count in object_counter.items():
        cv2.putText(
            frame, f"{obj}: {count}",
            (20, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2
        )
        y_offset += 25

    cv2.imshow("YOLO Real-Time Detection", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


**10: Release Camera & Windows**

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()

print("Camera stopped.")


**11: Save Results to Excel**

In [ ]:
df.to_excel("realtime_detections.xlsx", index=False)

print("Detections saved.")
print(f"Total detections: {len(df)}")


**12: Show Detection Table**

In [ ]:
df.head()
